# 웹스크래핑(Web Scrapping)

## 1. 개념

* 웹스크래핑(Web Scrapping) : 웹사이트상에서 특정부분에 위치한 정보를 컴퓨터로 자동추출하는 기능
* 웹크롤링(Web Crawling) : 자동화봇이 정해진 규칙에 따라 복수개의 웹페이지를 브라우징하는 기능

## 2. 필요한 라이브러리

* `BeautifulSoup` : HTML과 XML문선를 파싱하기 위한 라이브러리
* scrapy : Python을 작성된 오픈소스 웹크롤링 프레임워크

## 3. 스크래핑(클롤링)하는 방법

1. 원하는 페이지에 `요청(request)`을 보낸 후 그 결과를 html로 받는다.
1. 받은 html을 `파싱(Parsing)`처리를 한다.
1. 필요한 정보를 추출한다.
1. 파이썬으로 스크래핑을 하기 위해서는 `http request/response 모듈`과 `html을 파싱하는 모듈`이 필요하다.

##### 참고 사이트
* https://www.crummy.com/software/BeautifulSoup
* https://docs.python.org/3.0/library/urllib.request.html

### 1. 웹스크래핑 -  html 소스읽기

In [ ]:
# 1. html 소스읽기
from urllib.request import urlopen
html = urlopen('http://www.google.com')
print(type(html), html)
print(dir(html))
print()

html.read()

In [ ]:
# 2. 예외처리방법
from urllib.error import HTTPError
from urllib.error import URLError

try:
    html = urlopen('http://www.naverxx.com')
except HTTPError as e:
    print('HTTP 에러발생!!')
except URLError as e:    
    print('존재하지 않는 사이트 주소입니다!')
else:
    print(html.read())

In [ ]:
# 3. 이미지다운로드방법(1) - 간편한 방법
import urllib.request

# 다음사이트의 로고이미지를 다운로드하기
# https://t1.daumcdn.net/daumtop_chanel/op/20200723055344399.png
url = 'https://t1.daumcdn.net/daumtop_chanel/op/20200723055344399.png'
save_file = './data/daum.png'

urllib.request.urlretrieve(url, save_file)
print('파일이 정상적으로 다운로드 되었습니다!!')

In [ ]:
# 3. 이미지다운로드방법(2) - 바이너리파일로 처리
image = urllib.request.urlopen(url).read()
# print(type(image), image)

# 바이너리로 저장
with open('./data/daum_1.png', mode='wb') as f:
    f.write(image)
    print('파일이 정상적으로 다운로드 되었습니다!!')

In [ ]:
# 4. 매개변수를 추가해서 인터넷의 자료를 요청하기
# 기상청의 일기예보 사이트 : http://www.kma.go.kr
# 기상청육상중기예보
import urllib
from urllib.request import urlopen

API = 'https://www.weather.go.kr/weather/forecast/mid-term-rss3.jsp'

# url에 특수문자, 한글이 포함된 경우에는 URL인코딩이 필요
# 지역번호
#   전국 108, 서울/경기 109, 강원 105, 충북 131, 충남 133, 전북 146
#   전남 156, 경북 143, 경남 159, 제주 184

values = {'stnId':'108'} # 전국 날씨예보

# url인코딩
params = urllib.parse.urlencode(values)
print(type(params), params)

# 요청할 URL을 생성
url = API + '?' + params
print(url)

# 응답된 자료 읽기
data = urllib.request.urlopen(url).read() # 바이트타입으로 리턴
#  print(type(data), data)

# response데이터를 decoding
text = data.decode('utf-8')
print(text)

## 2. 웹스크래핑 - BeautifulSoup

* 다운로드 : https://www.crummy.com/software/BeautifulSoup
* `pip install beautifulsoup4` or `conda install -y beautifulsoup4`
* 주의) ~4를 생략하면 3.x버전이 설치 된다.

In [ ]:
!pip show beautifulsoup4

In [ ]:
# 1. BeautifulSoup : HTML을 파싱하는 Library
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs

# 1) url request
# https://www.google.com의 메이페이지의 타이틀 텍스트 가져오기
# url에 접속요청이 성공이 되면 HTTPResponse객체가 리턴된다.
html = urlopen('https://www.google.com')
print(type(html), html)

# 2) html.read() : HTTPResponse에서 html 데이터를 읽기
# BS의 기본분석기를 이용하여 html을 분석하기 위한 객체를 생성
# html분석기 -> html, xml분석기 -> xml
# html.read()
soup = bs(html.read(), 'html.parser')
# print(type(soup), soup)

# 문서내의 첫 h1태그를 선택
print(soup.h1)

In [ ]:
# 2. next_sibling

# 분석하고자하는 html
html ='''
<html><body>
    <h1>Hello Web Scraping</h1>
    <p>웹 페이지 분석</p>
    <p>웹 스크래핑</p>
</body></html>
'''
soup = bs(html, 'html.parser')
print(soup.h1, soup.p)

h1 = soup.html.body.h1
p1 = soup.html.body.p
print(h1, p1, type(p1))

# sibling 형제노드(동일레벨의 노드)
# previous_sibiling : 동일레벨의 이전노드
# next_sibling      : 동일레벨의 다음노드

p2 = p1.previous_sibling.previous_sibling 
# 2번하는 이유는 h1, \n
# 즉,  p1.previous_sibling -> \n(줄바꿈 문자) 선택
#      p1.previous_sibling.previous_sibling  -> h1 tag선택
print(p2)

p2 = p1.next_sibling   # 줄바꿈문자선책
p2 = p1.next_sibling.next_sibling # 2번째 p태그선택
print(p2)
print()

# Tag의 내부 text 추출하기
print(f'h1 = {h1.string}')
print(f'p1 = {p1.string}')
print(f'p2 = {p2.string}')

In [ ]:
# 3. find()
html ='''
<html><body>
    <h1 id="title">Hello Web Scraping</h1>
    <p id="body">웹 페이지 분석</p>
    <p>웹 스크래핑</p>
</body></html>
'''

soup = bs(html, 'html.parser')
# print(dir(soup))
# soup.find?
# 실습. id title인 태그, id가 body인 태그를 선택하고
# 내부문자열을 출력해보기
title = soup.find(id='title')
body = soup.find(id='body')
print(title, body)
print()

print(f'title = {title.string}')
print(f'body = {body.string}')

In [ ]:
# 4. find_all() : 특정속성을 가진 tag 전부를 추출하기
html ='''
<html><body>
    <li><a href="http://daum.net">daum</a></li>
    <li><a href="http://google.com">google</a></li>
    <li><a href="http://yahoo.com">yahoo</a></li>
    <li><a href="http://nate.com">nate</a></li>
    <li><a href="http://naver.com">naver</a></li>
</body></html>
'''

# 실습2. a태그 전체를 추출하기
# 전체를 가져올경우 데이터의 타입확인(bs4.element.ResultSet) 
# list자료형과 동일, list는 for문을 이용해서 출력

soup = bs(html, 'html.parser')

links = soup.find_all('a')
print(type(links), links)
print()

for link in links:
    # print(type(link), link)
    text = link.string
    # 태그내부의 속성 가져오기 : attrs
    href = link.attrs['href']
    print(f'{text} = {href}')

In [ ]:
# 5. CSS처리 : select_one(), select()
# 실습. h1의 도서목록을 추출하기 -> select_one() 
#       ... css선택자 : div#main > h1 
#       li목록 -> select()
#       ... css선택자 : div#main > ul > li
html ='''
<html><body>
    <div id="main">
        <h1>도서목록</h1>
        <ul>
            <li>자바프로그램 입문</li>
            <li>파이썬으로 하는 데이터분석</li>
            <li>HTML5/CSS3</li>
        </ul>
    </div>
</body></html>
'''
soup = bs(html, 'html.parser')
# soup.select?
# soup.select_one?

h1 = soup.select_one('div#main > h1').string
print(soup.h1.string, '=', h1)

# li목록 : select()
li_list = soup.select('div#main > ul > li')
print(type(li_list), li_list)
print()

for li in li_list:
    print(f'li = {li.string}')

#### 실습1. 기상청의 일기예보
![CDATA[○ (하늘상태, 강수) 이번 예보기간에는 전국이 가끔 구름많겠으나, 10일(금) 오전 제주도에 비가 오겠습니다. <br />○ (기온) 9일(목)~10일(금) 아침 기온은 4~15도, 낮 기온은 10~21도로 평년(최저기온 0~10도, 최고기온 11~18도)보다 조금 높겠으며,<br />          11일(토)~16일(목) 아침 기온은 -4~9도, 낮 기온은 7~17도로 평년과 비슷하거나 조금 낮겠습니다.<br />○ (주말전망) 11일(토)은 오전에 구름많다가 오후에 대체로 맑겠으나 강원영동과 제주도는 대체로 흐리겠습니다. 12일(일)은 중부지방은 대체로 맑겠고, 남부지방은 구름많겠습니다.<br />              아침 기온은 -2~9도, 낮 기온은 7~17도가 되겠습니다.]]

In [ ]:
import urllib.request as req 
from bs4 import BeautifulSoup as bs

stnId = input('지역번호를 입력하세요 => ')
url = f'https://www.weather.go.kr/weather/forecast/mid-term-rss3.jsp?stnId={stnId}'
print(url)

res = req.urlopen(url)
soup = bs(res, 'html.parser')

title = soup.find('title').string
print(title)
print()

wf = soup.find('wf').string
wf = wf.replace('<br />', '')
print(wf)

#### 실습2. 네이버금융에서 환율정보 추출하기

* http://finance.naver.com/marketindex
* 출력 usd/krw = 1302.20

In [ ]:
url = 'http://finance.naver.com/marketindex'
res = req.urlopen(url)
soup = bs(res, 'html.parser')
# <div class="head_info point_dn">
# <span class="value">1,302.20</span>

ex_rate = soup.select_one('div.head_info > span.value')
# print(ex_rate)
print(f'usd/krw = {ex_rate.string}')

In [ ]:
ex_rates = soup.select('div.head_info > span.value')
print(type(ex_rate), ex_rate)
print()
for rate in ex_rates:
    print(rate.string) # 통화별 환율구분해 보기

#### 실습3. 윤동주시인의 작품목록 추출하기

* https://ko.wikisource.org/wiki/저자:윤동주
  -- 한글주소상태로 복사하기 - 크롬웹스토어에서 `copy unicode urls` 설치
* select() 
* 웹브라우저에서 F12개발자도구, elements에서 원하는 부분을 우클릭
  - Copy > Copy selector : CSS 셀렉터

In [ ]:
url = 'https://ko.wikisource.org/wiki/%EC%A0%80%EC%9E%90:%EC%9C%A4%EB%8F%99%EC%A3%BC'
res = req.urlopen(url)
soup = bs(res, 'html.parser')

a_list = soup.select('.mw-parser-output > ul > li > a')
# print(type(a_list), len(a_list), a_list)

for a in a_list:
    name = a.string
    print(f'... {name}')


### 3. BeautifulSoup 관련함수

<!DOCTYPE html>
<html>
<head><title>The Dormouse's story</title></head>
<body>
<p class="title"><b>The Dormouse's story</b></p>
<p class="story">
    Once upon a time there were three little sisters; and their names were
    <a href="http://example.com/elsie" class="sister" id="link1">Elsie</a>,
    <a href="http://example.com/lacie" class="sister" id="link2">Lacie</a> and
    <a href="http://example.com/tillie" class="sister brother" id="link3">Tillie</a>;
    and they lived at the bottom of a well.
</p>
</body>
</html>

In [ ]:
import urllib.request as req 
from bs4 import BeautifulSoup as bs

In [ ]:
html = '''
    <!DOCTYPE html>
    <html>
    <head><title>The Dormouse's story</title></head>
    <body>
    <p class="title"><b>The Dormouse's story</b></p>
    <p class="story">
        Once upon a time there were three little sisters; and their names were
        <a href="http://example.com/elsie" class="sister" id="link1">Elsie</a>,
        <a href="http://example.com/lacie" class="sister" id="link2">Lacie</a> and
        <a href="http://example.com/tillie" class="sister brother" id="link3">Tillie</a>;
        and they lived at the bottom of a well.
    </p>
    </body>
    </html>
'''
soup = bs(html, 'html.parser')

# 1. element
print(soup.title, soup.find('title'))

# 2. tag 이름
print(soup.title.name, soup.find('title').name)

# 3. text
print(soup.title.string, soup.find('title').string, soup.title.get_text())

# 4. single element
print(soup.a.get_text(), soup.find('a').string, soup.find('a').get_text())

# 5. multi element
print(soup.find_all('a'))
print(soup.find_all('a')[0].get_text(),soup.find_all('a')[1].get_text(),soup.find_all('a')[2].get_text())

# 6. attribute
print(soup.a['class'], soup.a.get('class'))
print(soup.a['href'], soup.a.get('href'))

# 7. find by id
print(soup.find(id='link1'))
print(soup.find('a', {'id':'link1'}))
print()

# 8. find by class
print(soup.find(class_='sister'))
print(soup.find('a', {'class':'sister'}))
print(soup.find_all('a', {'class':'sister'}))

#### 4. 고급 - HTML 분석하기

* https://github.com/REMitchell/python-scraping

In [ ]:
# 1. 전쟁과 평화
html = req.urlopen('http://pythonscraping.com/pages/warandpeace.html')
soup = bs(html, 'html.parser')

In [ ]:
# 실습 1) span태그에서 class가 green인 태그 : 등장인물의 이름을 출력
# findAll(), find_all()
names = soup.findAll('span', {'class':'green'})
for name in names:
    print(name.get_text())

In [ ]:
# 실습 2) 웹페이지에서 모든 heading태그(h1~h6)의 text를 출력
#         - list안에 for문을 사용
titles = soup.find_all(['h1','h2','h3','h4','h5','h6',])
print([title for title in titles])
print([title.string for title in titles])
print([title.get_text() for title in titles])

In [ ]:
# 실습 3) span태그에서 class가 green or red인 태그의 text출력
#         - find_all('', {'class': {집합}), list안에 for문
all_text = soup.find_all('span', {'class': ['green', 'red']})
print([text.string for text in all_text])

In [ ]:
# 실습 4) wordcount = 'the prince'단어의 갯수를 출력
wordcount = soup.find_all(text='the prince')
print(type(wordcount), len(wordcount), wordcount)
print(f'the prince의 출현갯수 = {len(wordcount)}')

In [ ]:
# 자식노드 추출하기
html = req.urlopen('http://pythonscraping.com/pages/page3.html')
soup = bs(html, 'html.parser')

In [ ]:
# 실습 5) table태그중에서 id가 giftList인 태그의 자식노드를 추출
# hint) 자식노드 : soup.find().children, soup.find().tr.previous_sibling, next...
a = soup.find('table', {'id':'giftList'}).children
print(type(a))

for child in soup.find('table', {'id':'giftList'}).children:
    print(child)

In [ ]:
# 실습 6) next_siblings
# table태그에 id giftList인 태그의 자식노드 추출
# hint) soup.find().tr.next_siblings
for sibling in soup.find('table', {'id':'giftList'}).tr.next_siblings:
    print(sibling)

In [ ]:
# 실습 7) previous_sibling
# img1.jpg의 부모노드의 이전형제노드의 text추출하기
# sound.find().parent.previous_sibling을 이용
print(soup.find('img', {'src':'../img/gifts/img1.jpg'}))
print(soup.find('img', {'src':'../img/gifts/img1.jpg'}).parent.previous_sibling)
print(soup.find('img', {'src':'../img/gifts/img1.jpg'}).parent.previous_sibling.string)
print(soup.find('img', {'src':'../img/gifts/img1.jpg'}).parent.previous_sibling.get_text())

In [ ]:
# 실습8) 정규식을 사용 - img태그 전체를 추출하기
# re.complie(../img/gifts/img*.jpg)
import re
imgs = soup.find_all('img', {'src': re.compile('\.\.\/img\/gifts\/img.*\.jpg')})

for img in imgs:
    print(img['src'])

In [ ]:
# 실습 9) lambda식이용

# a. tag속성이 2개인 것만 추출
print(soup.find_all(lambda tag: len(tag.attrs)>=2))
print()

# b. text = 'Or maybe he\'s only resting?'만 추출
print(soup.find_all(lambda tag: tag.get_text() == 'Or maybe he\'s only resting?'))
print(soup.find_all('', text='Or maybe he\'s only resting?'))

### 5. json분석

In [ ]:
# json탐색 : {key:value}
import json
json_str = '''{
    "amount":[{"num":0},{"num":1},{"num":2}], 
    "fruits":[{"fruit":"apple"},{"fruit":"banana"},{"fruit":"pear"}]
}'''
print(type(json_str), json_str)

# json을 dict 변환
dict_data = json.loads(json_str) 
print(type(dict_data), dict_data)
print()

print(type(dict_data.get('fruits')), dict_data.get('fruits'))
print(dict_data.get('fruits')[0])
print(dict_data.get('fruits')[1])
print(dict_data.get('fruits')[2])

### 6. pdf 분석

* `pip install pdfminer3k`

In [ ]:
!pip install pdfminer3k

In [ ]:
!pip show pdfminer3k

In [ ]:
from pdfminer.pdfinterp import PDFResourceManager, process_pdf
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from io import StringIO
from io import open

In [ ]:
import urllib.request as req 
from bs4 import BeautifulSoup as bs

In [ ]:
def readPDF(pdfFile):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, laparams=laparams)
    
    process_pdf(rsrcmgr, device, pdfFile)
    device.close()
    
    content = retstr.getvalue()
    retstr.close()
    
    return content

In [ ]:
# 전쟁과 평화
pdfFile = req.urlopen('https://pythonscraping.com/pages/warandpeace/chapter1.pdf')
print(type(pdfFile), pdfFile)

content = readPDF(pdfFile)
print(type(content), content)
pdfFile.close()

#### 7. Data저장

In [ ]:
# 웹페이지의 내용을 분석하여 csv파일로 저장
# table태그의 내부 텍스트를 저장
import csv

html = req.urlopen('http://en.wikipedia.org/wiki/Comparison_of_text_editors')
soup = bs(html, 'html.parser')

# class가 'wikitable'인 태그중에서 첫 번째 tag를 선택
table = soup.findAll('table', {'class': 'wikitable'})[0]
# print(type(table), table)
rows = table.findAll('tr')
# print(rows)

csv_file = open('./data/web/editors.csv', 'wt', newline='', encoding='utf-8')
writer = csv.writer(csv_file)

try:
    for row in rows:
        csv_row = []
        # td, th태그이 내용을 list에 추가
        for cell in row.findAll(['td', 'th']):
            csv_row.append(cell.get_text())
        writer.writerow(csv_row)
except Exception as e:
    print('파일입출력에 에러가 발생했습니다!')
finally:
    print('파일이 성공적으로 저장이 되었습니다!')
    csv_file.close()

In [ ]:
from urllib.request import urlretrieve, urlopen
from bs4 import BeautifulSoup as bs

html = urlopen('http://www.pythonscraping.com')
soup = bs(html, 'html.parser')

img = soup.find('a', {'id':'logo'}).find('img')['src']
urlretrieve(img, 'logo.jpg')

연습문제 1.
다음 사이트에서 링크가 되어 있는 모든 제목을 가져와서 출력하기
* http://media.daum.net/digital/

연습문제 2.
네이버사이트 이미지 검색후 '../data/web'에 한번에 다운로드 및 저장하기(10건정도)
* 검색어를 전달받아서 결과를 저장
* https://search.naver.com/search.naver?sm=tab_hty.top&where=image&query=코로나

#### 8. 웹사이트정보를 mysql로 저장

##### 1. mysql database와 table 생성
```sql
create database pyweb default character set utf8;

create table pages(
    id int not null auto_increment primary key,
    title varchar(200),
    content text,
    created timestamp default current_timestamp
);  
```
##### 2. mysql 연결하기
>* 설치 : 
>>* pip install pymysql 또는
>>* conda install pymysql
* 연결 : `pymysql.connect(host='localhost', port=3306, user='root', passwd='12345', db='pyweb', charset='utf8')`

In [ ]:
from bs4 import BeautifulSoup as bs
import urllib.request as req
import re
import datetime
import random
import pymysql

In [ ]:
conn = pymysql.connect(host='localhost', port=3306
                     , user='root', passwd='12345'
                     , db='pyweb', charset='utf8')
conn

In [ ]:
cursor = conn.cursor()
cursor.execute('use pyweb')
cursor.execute('delete from pages')
conn.commit()

In [ ]:
random.seed(42)

# 테이블에 데이터를 저장하는 함수
def store(title, content):
    # 따옴표처리
    title = title.replace("'", "''") # 작은 따옴표1개를 2개로 치환
    title = title.replace('"', "\'") # 큰 따옴표는 \'로 치환
    content = content.replace("'", "''")
    content = content.replace('"', "\'")
    content = content.replace('\n', '') # 줄바꿈문자는 제거
    
    sql = 'insert into pages(title, content) values("%s", "%s")' % (title,content)
    cursor.execute(sql)
    conn.commit()   

In [ ]:
# 웹사이트를 링크
def getLinks(url):
    html = req.urlopen('https://en.wikipedia.org' + url)
    soup = bs(html, 'html.parser')
    title = soup.find('h1').get_text()
    content = soup.find('div', {'id':'mw-content-text'}).find('p').get_text()
    store(title, content)
    
    # ^: 시작, $: 끝, *: 반복
    # ^(/wiki/) : /wiki/ 로 시작되는 a 태그를 찾기
    # ((?!:).) : .로 끝나는 문자에서 ?,!,:를 검색
    # 전체의미는 /wiki/ + 문자열 + .의 패턴에서 문자열중에 ?,!,:를 검색
    return soup.find('div', {'id':'bodyContent'}) \
               .findAll('a', href=re.compile("^(/wiki/)((?!:).)*$"))

In [ ]:
conn = pymysql.connect(host='localhost', port=3306
                     , user='root', passwd='12345'
                     , db='pyweb', charset='utf8')

cursor = conn.cursor()
cursor.execute('use pyweb')
cursor.execute('delete from pages')
conn.commit()

links = getLinks('/wiki/Kevin_Bacon')
print(links, len(links))

In [ ]:
try:
    count = 0
    while len(links) > 0:
        idx = random.randint(0, len(links) - 1)
        newArticle = links[idx].attrs['href']
        print(newArticle, idx)
        link = getLinks(newArticle)
        count += 1
        if count >=5:
            break
except Exception as e:
    print('저장실패!!!')
else:
    print('저장성공!!')
finally:
    cursor.close()
    conn.close()
    

#### 9. 웹사이트 캡춰

##### 1. phantomjs

1. http://phantomjs.org/download.html
1. phantomjs-2.1.1-windows.zip
1. 04.python\phantomjs에 압축해제(

##### 2. selenium

1. `pip install selenium` or `conda install -y selenium`
1. 웹브라우저를 컨트롤하여 자동화시켜주는 도구
   - `webdriver`라는 API를 통해 운영체제에 설치된 chrome등의 웹브라우저를 제어
1. 제공하는 기능
   - url에 접근하는 api : `get(url)`
   - 페이지의 단일 엘리먼트에 접근하는 api
     - `find_element_by_id(html_id)`
     - `find_element_by_name(html_name)`
     - `find_element_by_xpath(html/body/div/img)`
   - 페이지의 복수 엘리먼트에 접근하는 api
     - `find_element_by_css_selector('#id > div.selector')`
     - `find_element_by_css_name('클래스명')`
     - `find_element_by_tag_name('h1')`
1. selenium을 사용하려면 해당 웹브라우저의 webdriver를 다운해야 한다.
   - 웹브라우저에 맞는 드라이버를 설치 해야 한다.
     - 크롬의 경우 웹브라우저의 버전을 확인 -> 119.????,??
     - 119버전에 맞는 webdriver(chromedeiver.exe)를 다운
       -https://googlechromelabs.github.io/chrome-for-testing/#stable
   - Chrome : https://sites.google.com/a/chromium.org/chromedriver/downloads
   - Edge : https://developer.microsoft.com/en-us/microsoft-edge/tools/webdriver/
   - Firefox : https://github.com/mozilla/geckodriver/releases
   - 크롬만 다운
     - 04.python에 압축해제
     
1. selenium 4.x 버전
   * https://www.selenium.dev/documentation/webdriver/troubleshooting/upgrade_to_selenium_4/
   * 참고사이트 : * https://blog.naver.com/hi_eng_/222928621005

In [ ]:
!pip install selenium         # 최신버전 설치
# !pip install selenium==3.8.0  # 특정 버전을 설치
# !pip install --upgrade selenium  # 현재버전에서 최신버전
!pip show selenium

In [ ]:
!pip install pyperclip
!pip show pyperclip

In [ ]:
USER = '네이id'
PASS = '네이버PW'

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pyperclip

In [ ]:
driver = webdriver.Chrome()  # 크롬웹브라우저를 생성
driver.get('http://www.naver.com') # 로그인하기 위한 사이트주소
time.sleep(3)   # 3초지연(대기시간)

# 로그인버튼 클릭
login_btn = driver.find_element(By.CLASS_NAME, 'MyView-module__link_login___HpHMW')
login_btn.click()
time.sleep(3)

# id, pw를 clear 
tag_id = driver.find_element(By.ID, 'id')
tag_pw = driver.find_element(By.ID, 'pw')
tag_id.clear()
tag_pw.clear()
time.sleep(1)

# id, pw를 자동으로 입력
tag_id.click() # focus이동
pyperclip.copy(USER)                  # 클립보드로 카피
tag_id.send_keys(Keys.CONTROL, 'v')   # ctrl+v 클릭
time.sleep(1)

tag_pw.click() # focus이동
pyperclip.copy(PASS)                  # 클립보드로 카피
tag_pw.send_keys(Keys.CONTROL, 'v')   # ctrl+v 클릭
time.sleep(1)

# 로그인버튼클릭
login_btn = driver.find_element(By.ID, 'log.login')
login_btn.click()

In [ ]:
# 실습. 네이버쇼핑 구매내역조회하기
driver.get("https://order.pay.naver.com/home?tabMenu=SHOPPING")
prod_lists = driver.find_element(By.CLASS_NAME, 'PaymentList_link-detail__I74mH')
print(type(prod_lists), prod_lists)

# for product in prod_lists:
#     pname = product.find_element(By.CSS_SELECTOR, 'p').text
#     print(pname)

##### 실습1. 교보문고에서 도서목록 수집하기

* url : https://search.kyobobook.co.kr/web/search?vPstrKeyWord={url.parse.quote(book_name)}
  - 
* 책 제목이 파이썬인 도서를 출력하세요!
>22권이 검색되었습니다
>- 혼자 공부하는 파이썬
>- Do it! 점프 투 파이썬
>- 두근두근 파이썬
> Python Essential Reference, 2/e

In [6]:
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs
import re

# 교보사이트에서 python검색
html = urlopen('https://search.kyobobook.co.kr/search?keyword=python&gbCode=TOT&target=total')
soup = bs(html, 'html.parser')

# select()
# #shopData_list > ul > li:nth-child(1) > div > div.prod_info_box > span > a > span:nth-child(2)
titles = soup.select('a.prod_info > span')
# print(type(titles), titles)

print(f'{len(titles)} 권이 검색되었습니다!')
print('-' * 60)

# print(titles)

for title in titles:
    if(title.text[0] !='[' and title.text[0] !='예'):
        print(title.text)

58 권이 검색되었습니다!
------------------------------------------------------------
파이썬 텍스트 마이닝 바이블 1
파이썬 텍스트 마이닝 바이블 2
코딩 테스트 합격자 되기: 파이썬 편
작심 3일 파이썬 Python
2023 빅데이터분석기사 실기 한권완성 파이썬(Python)
초보자를 위한 파이썬(Python) 200제
Getting Start Python(파이썬)
파이썬 딥러닝 파이토치(Python Deep Learning PyTorch)
파이썬(Python) 3학년 머신러닝의 구조
파이썬 클린 코드
고성능 파이썬
러닝 파이썬(하)
러닝 파이썬(상)
전문가를 위한 파이썬 프로그래밍
처음 시작하는 파이썬
Black Hat Python
실전 파이썬 핸즈온 프로젝트
파이썬 매일 코딩
파이썬과 수치 해석
파이썬으로 공부하는 블록체인


In [8]:
import urllib
def book_search(book_name):
    html = urlopen(f'https://search.kyobobook.co.kr/web/search?vPstrKeyWord={urllib.parse.quote(book_name)}')
    soup = bs(html, 'html.parser')   
    
    books = soup.select('a.prod_info > span')
    print(f'{len(books)} 권이 검색되었습니다!')
    print('-' * 60)
    
    for book in books:
        if(book.text[0] !='[' and book.text[0] !='예'):
            print(book.text)
            
book_name = input('검색할 도서명을 입력하세요 => ')
book_search(book_name)

검색할 도서명을 입력하세요 => 머신러닝
55 권이 검색되었습니다!
------------------------------------------------------------
머신 러닝(Machine Learning)
이보다 더 쉬울 수 없는 자바 머신러닝 with Weka
핸즈온 머신러닝
실전! 컴퓨터비전을 위한 머신러닝
머신 러닝 교과서 with 파이썬, 사이킷런, 텐서플로
파이썬 라이브러리를 활용한 머신러닝
금융 전략을 위한 머신러닝
그래프 머신러닝
살아 움직이는 머신러닝 파이프라인 설계
R을 활용한 머신러닝
그림으로 배우는 StatQuest 머신러닝 강의
머신러닝 디자인 패턴
머신러닝 파워드 애플리케이션
비즈니스 머신러닝
핸즈온 머신러닝
사이버 보안을 위한 머신러닝 쿡북
자산운용을 위한 금융 머신러닝
AWS 클라우드 머신러닝
통계학으로 배우는 머신러닝
금융 전문가를 위한 머신러닝 알고리즘


##### 연습문제. 본인이 scraping하는 사이트를 선정해서 검색추출결과를 출력하기